In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

## Задача

Для данных о погоде создать категорию "время года" и закодировать ее при помощи OneHotEncoder. Проще всего применить OneHotEncoder при помощи `pandas.get_dummies()`

Преобразовать данные о температуре двумя способами.

In [11]:
df = pd.read_csv('data/weather.csv', index_col=0, parse_dates=True)

mapper = {
    1: 'winter',
    2: 'winter',
    3: 'spring',
    4: 'spring',
    5: 'spring',
    6: 'summer',
    7: 'summer',
    8: 'summer',
    9: 'autumn',
    10: 'autumn',
    11: 'autumn',
    12: 'winter'
}

df['month'] = df.index.month
df['season'] = df['month'].map(mapper)
pd.concat([df, pd.get_dummies(df['season'])], axis=1).head()

,t,month,season,autumn,spring,summer,winter
Day,,,,,,,
2008-01-01,0,1,winter,0,0,0,1
2008-01-02,-5,1,winter,0,0,0,1
2008-01-03,-11,1,winter,0,0,0,1
2008-01-04,-11,1,winter,0,0,0,1
2008-01-05,-12,1,winter,0,0,0,1


## Задача

Перебрать параметры `n_estimators`, `criterion` и `max_depth` и найти оптимальные. 

Для решения можно написать тройной цикл, в каждом из которых можно перебирать значения параметров.

In [2]:
data = load_digits()
x = data.data
y = data.target
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [14]:
def get_accuracy_of_model(x_train, y_train, x_test, y_test, n_estimators, criterion, max_depth):
    """Функция принимает на вход треин и тест выборки и параметры модели. На выходе дает accuracy."""
    model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return accuracy_score(y_test, y_pred)

**Вариант 1. Перебираем в тройном цикле**

In [18]:
n_estimators = [5, 100, 300, 500]
criterions = ['gini', 'entropy']
max_depths = [3, 5, 7, 10]

best_acc = 0
best_params = None
# Ваш код здесь
for n in n_estimators:
    for criterion in criterions:
        for max_depth in max_depths:
            acc = get_accuracy_of_model(x_train, y_train, x_test, y_test, n, criterion, max_depth)
            if acc > best_acc:
                best_acc = acc
                best_params = (n, criterion, max_depth)

best_acc, best_params

(0.96, (100, 'entropy', 10))

**Вариант 2. Генерируем сетку и перебираем по ней**

In [19]:
import itertools

best_acc = 0
best_params = None
for (n, criretion, max_depth) in itertools.product(n_estimators, criterions, max_depths):
    acc = get_accuracy_of_model(x_train, y_train, x_test, y_test, n, criterion, max_depth)
    if acc > best_acc:
        best_acc = acc
        best_params = (n, criterion, max_depth)
        
best_acc, best_params

(0.96, (100, 'entropy', 10))

## Задача

Построить модель SGDClassifier и осуществить подбор гиперпараметров при помощи случайного поиска (список гиперпараметров можно посмотреть в документации).

In [22]:
params = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.00001, 0.0001, 0.1],
    'max_iter': [100, 1000, 2000],
    'eta0': [0, 0.1, 0.0001]
}

r_search = RandomizedSearchCV(SGDClassifier(), params)
r_search.fit(x_train, y_train)
r_search.best_estimator_

C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter t

SGDClassifier(eta0=0.0001, max_iter=100, penalty='elasticnet')

In [23]:
r_search.best_score_

0.9547046674927715

## Задача

Обучить модель градиентного бустинга на датасете, провести подбор гиперпараметров (взять любые 3 штуки, их список можно получить через документацию к модели).

Сравнить качество с полученными ранее результатами.

In [25]:
params = {
    'loss': ['deviance', 'exponential'],
    'learning_rate': [0.0001, 0.00001, 0.0001, 0.1],
    'n_estimators': [100, 300, 500],
    'max_depth': [2, 3, 5]
}

r_search = RandomizedSearchCV(GradientBoostingClassifier(), params)
r_search.fit(x_train, y_train)
r_search.best_estimator_

C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\ensemble\_gb.py", line 525, in fit
    self._check_params()
  File "C:\Users\koval\anaconda3\envs\data-science-class\lib\site-packages\sklearn\ensemble\_gb.py", line 310, in _chec

GradientBoostingClassifier(n_estimators=500)

В warning сказано, что 30 из 50 моделей не смогли обучиться, потому что лосс-функция `exponential` работает только с бинарной классификацией. Можно убрать подбор по лоссу, и тогда отработает быстрее и качественнее.

In [26]:
r_search.best_score_

0.9643563265868099

## Задача

Иногда нужно использовать модели, у которых интерфейс отличается от интерфейса моделей из sklearn и тогда приходится самостоятельно реализовывать стекинг.

Задача 

Алгоритм построения стекинга:
- Получить прогнозы базовых моделей
- Использовать прогнозы базовых моделей в качестве признаков для финализирующей модели
- Оформить модель в отдельную функцию

In [3]:
def preprocess_data(x):
    return (x - x.mean()) / x.std()

In [26]:
def fit_models(x_train, y_train, models):
    """Обучаем модели"""
    for model in models:
        model.fit(x_train, y_train)

def predict_base_models(x, models):
    """Предсказываем и нормируем прогнозы"""
    res = []
    for model in models:
        res.append(model.predict(x))
    base_predictions = np.array(res).T
    # Некоторые модели чувствительны к тому, чтобы на входе были нормированные данные
    return preprocess_data(base_predictions)

def predict_proba_base_models(x, models):
    """Предсказываем через predict_proba (у меня результат получался хуже на этих данных)"""
    res = np.array([])
    for model in models:
        if len(res) == 0:
            res = model.predict_proba(x)
        else:
            res = np.hstack((res, model.predict_proba(x)))
    return res

In [85]:
def fit_stacking(x_train, y_train, models, final_model):
    fit_models(x_train, y_train, models)
    final_features = predict_base_models(x_train, models)
    final_model.fit(final_features, y_train)
    return models, final_model

In [86]:
def predict_stacking(x, models, final_model):
    final_features = predict_base_models(x, models)
    return final_model.predict(final_features)

Для лучшей сходимости методов преобразуем данные

In [87]:
x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

Используем модели из предыдущих заданий

In [88]:
# Выбираем модели
models = [
    RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7),
    LogisticRegression(penalty='elasticnet', max_iter=1000, solver='saga', l1_ratio=0.5)
]
final_model = DecisionTreeClassifier(max_depth=12)

# Обучаем стекинг
models, final_model = fit_stacking(x_train, y_train, models, final_model)

In [89]:
y_pred = predict_stacking(x_test, models, final_model)
accuracy_score(y_test, y_pred)

0.9755555555555555

Точность немного выше, чем было до этого